# Análise de Avaliações de Hospedagem em Nova York

Este notebook oferece uma análise detalhada das avaliações de hospedagens em Nova York, explorando estatísticas descritivas, tendências de revisões ao longo do tempo, e destacando as melhores e piores hospedagens com base em uma variedade de critérios.

In [5]:
# Arredondar 'price' para 1 casa decimal e 'minimum_nights' para 0 casas decimais
descriptive_stats_rounded = listings[['price', 'minimum_nights']].describe().round({'price': 1, 'minimum_nights': 0})
descriptive_stats_rounded

,price,minimum_nights
count,23761.0,38377.0
mean,206.8,29.0
std,762.3,30.0
min,10.0,1.0
25%,80.0,30.0
50%,135.0,30.0
75%,230.0,30.0
max,100000.0,1250.0


In [17]:
# Verificar se a coluna 'last_review' pode ser convertida para datetime
listings['last_review'] = pd.to_datetime(listings['last_review'], errors='coerce')

# Criar colunas 'year' e 'month' a partir de 'last_review'
listings['year'] = listings['last_review'].dt.year
listings['month'] = listings['last_review'].dt.month

# Agrupar por ano e mês e contar o número de avaliações
reviews_by_year_month = listings.groupby(['year', 'month']).size().reset_index(name='reviews_count')

# Remover linhas com ano ou mês nulos
reviews_by_year_month.dropna(subset=['year', 'month'], inplace=True)

# Converter ano e mês para inteiro para facilitar a visualização
reviews_by_year_month['year'] = reviews_by_year_month['year'].astype(int)
reviews_by_year_month['month'] = reviews_by_year_month['month'].astype(int)

# Criar uma coluna 'year_month' para o eixo x do gráfico
reviews_by_year_month['year_month'] = reviews_by_year_month['year'].astype(str) + '-' + reviews_by_year_month['month'].astype(str).str.zfill(2)

# Gráfico de linhas para mostrar a evolução do número de avaliações ao longo do tempo
fig_reviews_evolution = px.line(reviews_by_year_month, x='year_month', y='reviews_count', title='Evolução do Número de Avaliações por Ano e Mês')
fig_reviews_evolution.update_xaxes(title_text='', showticklabels=False, showgrid=False, zeroline=False)
fig_reviews_evolution.update_yaxes(title_text='', showticklabels=False, showgrid=False, zeroline=False)
fig_reviews_evolution.show()

In [9]:
# Identificar os anúncios com as melhores e piores avaliações
best_reviews = listings.sort_values(by='number_of_reviews', ascending=False).head()
worst_reviews = listings.sort_values(by='number_of_reviews', ascending=True).head()

# Identificar os bairros mais comentados
neighbourhood_reviews = listings.groupby('neighbourhood')['number_of_reviews'].sum().sort_values(ascending=False).head()

# Exibir os resultados
best_reviews[['name', 'number_of_reviews', 'neighbourhood']], worst_reviews[['name', 'number_of_reviews', 'neighbourhood']], neighbourhood_reviews

(                                                    name  ...     neighbourhood
 15156  Amazing Micro Unit W/ communal rooftop and kit...  ...      East Village
 20012  Come catch a Broadway Show & stay in Times Square  ...  Theater District
 21129               Untitled at 3 Freeman - Studio Queen  ...   Lower East Side
 20041      Short walk to Central Park/5th Ave/Radio City  ...  Theater District
 21284        Steps away from Central Park/Hell's Kitchen  ...  Theater District
 
 [5 rows x 3 columns],
                                                     name  ...       neighbourhood
 19188                 Cozy  Studio Place Away From Home.  ...  Bedford-Stuyvesant
 25002                             Semi Basement Room #13  ...         Brownsville
 25001          FiDi Studio w/ Office by Anyplace - 11417  ...  Financial District
 25000          FiDi Studio w/ Office by Anyplace - 11418  ...  Financial District
 24976  Park Slope 2 BR + Office Duplex w/large yard/d...  ...         Sou

A análise do código Python foca em dados de anúncios de hospedagem, dividida em três partes principais:

1. **Análise de Avaliações:**
   - `best_reviews`: Seleciona os 5 anúncios com mais avaliações, indicando popularidade.
   - `worst_reviews`: Escolhe os 5 anúncios com menos avaliações, sugerindo menor interesse.

2. **Bairros Populares:**
   - `neighbourhood_reviews`: Agrupa anúncios por bairro, somando e ordenando as avaliações para identificar os 5 bairros mais comentados.

3. **Resultados:**
   - Exibe informações dos anúncios mais e menos populares e lista os bairros mais comentados.

Essa análise ajuda tanto anfitriões quanto hóspedes a entender quais anúncios e bairros são mais atraentes na plataforma.

In [52]:
import plotly.express as px

# Gráfico de barras horizontais para os bairros mais comentados, sem linhas de grade e sem valores do eixo x
neighbourhood_reviews = listings.groupby('neighbourhood')['number_of_reviews'].sum().sort_values(ascending=True).tail(10)
fig_neighbourhood_reviews = px.bar(neighbourhood_reviews, y=neighbourhood_reviews.index, x=neighbourhood_reviews.values, 
                                   title='Bairros mais avaliados',
                                   labels={'y': 'Bairro', 'x': 'Número Total de Comentários'},
                                   orientation='h',
                                   text_auto='.2s')  # Adiciona o total das linhas y por extenso
fig_neighbourhood_reviews.update_layout(xaxis_showgrid=False, yaxis_showgrid=False, xaxis={'visible': False})  # Remove as linhas de grade e os valores do eixo x
fig_neighbourhood_reviews.show()